In [1]:
import numpy as np

# numpy settings
np.random.seed(1)
np.set_printoptions(suppress=True, linewidth = 150)

In [2]:
# MNIST dataset
from dataload import read_images_labels

k = 10 # k-hot value

# load train
training_images_filepath = './data/train-images-idx3-ubyte/train-images-idx3-ubyte'
training_labels_filepath = './data/train-labels-idx1-ubyte/train-labels-idx1-ubyte'
x_train, y_train = read_images_labels(training_images_filepath, training_labels_filepath)

# load test
test_images_filepath = './data/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'
test_labels_filepath = './data/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte'
x_test, y_test = read_images_labels(test_images_filepath, test_labels_filepath)

print(np.array(x_train[45854]).reshape(28,28))
print(np.array(y_train[45854]))

# reformat data for model
x_train = x_train.transpose() * (1/255)

# data dimension D
D = x_train.shape[0]
assert D==784
# number of training examples N
N = x_train.shape[1]

# reformat data to k-hot format TODO: does numpy have a better way to do this?
temp_array = np.zeros((k, N))
for index,val in enumerate(y_train):
    temp_array[val][index] = 1
print(temp_array[:, 45854])
y_train = temp_array

x_test = x_test.transpose() * (1/255)
y_test = y_test.reshape(1,-1)

ModuleNotFoundError: No module named 'dataload'

In [ ]:
print(x_train.shape)

In [ ]:
# iris dataset

k = 3 # k-hot value

# csv columns are sepal_length,sepal_width,petal_length,petal_width,species
iris_dataset = np.genfromtxt('iris.csv', delimiter=',')

# randomize and split
np.random.seed(1)
np.random.shuffle(iris_dataset)

data = iris_dataset[:, [range(4)]]
labels = iris_dataset[:, 4]

data = np.reshape(data, (150,4))

x_train = data[[range(iris_dataset.shape[0]//5*3)]]
x_train = x_train.transpose()
x_test = data[[range(iris_dataset.shape[0]//5*3, iris_dataset.shape[0])]]
x_test = x_test.transpose()

# data dimension D
D = x_train.shape[0]
assert D==4
# number of training examples N
N = x_train.shape[1]

labels_train = labels[[range(iris_dataset.shape[0]//5*3)]]

# format
# reformat data to k-hot format TODO: does numpy have a better way to do this?
y_train = np.zeros((k, N))
for index, val in enumerate(labels_train): y_train[int(val)][index] = 1

y_test = labels[iris_dataset.shape[0]//5*3:iris_dataset.shape[0]].reshape(1,-1)


In [10]:
### instatiate model architecture

# architecture hyperparameters
hidden_dim = 15 # width of hidden layer

# instatiate first matrix of weights
w_2 = np.random.normal(loc=0, scale=1, size=(hidden_dim,D))
# instatiate first bias vector
b_2 = np.random.normal(loc=0, scale=1, size=(hidden_dim,1))

# instatiate second set of weights
w_3 = np.random.normal(loc=0, scale=1, size=(k,hidden_dim))
# instatiate second set of weights
b_3 = np.random.normal(loc=0, scale=1, size=(k,1))

print(w_3)

def sigmoid(x): return 1/(1+np.exp(-x))

def sigmoid_prime(x): 
    sig = sigmoid(x)
    return sig * (1-sig)

# implementation of softmax from: https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python 
def softmax(x): 
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

### training

# training hyperparamters
alpha = 0.1 # learning rate
epochs = 3 # number of epochs
batch_size = 5

def extrem(array, array_name): return f'Extremal Values for {array_name}: {np.amin(array), np.amax(array)}'

def loss(activation, label): return 0.5*(activation-label)**2

def cost(loss_vec): return np.average(loss_vec)

for epoch in range(epochs):
    for batch_index in range(int(N/batch_size)): # TODO look at this again
        x_train_batch = x_train[:, [batch_index, batch_index + batch_size]] 
        y_train_batch = y_train[:, [batch_index, batch_index + batch_size]]

        # forward pass
        z_2 = w_2.dot(x_train_batch) + b_2.dot(np.ones((1,x_train_batch.shape[1])))
        a_2 = sigmoid(z_2) 
        z_3 = w_3.dot(a_2) + b_3.dot(np.ones((1,a_2.shape[1])))
        a_3 = sigmoid(z_3)

        # backprop
        delta_3 = np.multiply((a_3-y_train_batch), sigmoid_prime(z_3))
        delta_2 = np.multiply(w_3.transpose().dot(delta_3), sigmoid_prime(z_2))

        w_3 -= alpha*(1/batch_size)*delta_3.dot(a_2.transpose())
        b_3 -= alpha*np.reshape(np.mean(delta_3, axis=1), (-1,1))

        w_2 -= alpha*(1/batch_size)*delta_2.dot(x_train_batch.transpose())
        b_2 -= alpha*np.reshape(np.mean(delta_2, axis=1), (-1,1))
        # if batch_index % 10 == 0: print(f"Batch {batch_index} completed.")

    # final foward pass
    a_2 = sigmoid(w_2.dot(x_train) + b_2.dot(np.ones((1,x_train.shape[1]))))
    a_3 = sigmoid(w_3.dot(a_2) + b_3.dot(np.ones((1,x_train.shape[1]))))

    loss_val = loss(a_3, y_train)
    cost_val = cost(loss_val)
    if epoch % 100 == 0: print(f"Cost at epoch {epoch}: {cost_val}")
    # print(f"Cost at epoch {epoch}: {cost_val}")


[[ 0.437 -0.096  0.834 -1.575 -1.011 -1.3    0.325  0.512 -0.379  0.094 -0.93   0.224  1.136  1.052  0.033]
 [-0.506 -1.388  1.11  -1.15   0.161  0.736  0.645 -0.695 -0.336 -0.827 -0.217  1.464  0.434 -0.172 -1.058]
 [-0.486  0.283  0.504  1.224  0.253  0.422  0.366  0.896  1.678 -0.459  0.277 -2.194  0.991  0.097  0.211]
 [-0.124  0.199  0.853 -0.087 -1.5   -0.799  1.62   0.828  1.223  0.501 -0.074 -0.528 -0.508 -1.332 -0.225]
 [ 0.549 -1.225 -0.797  0.206  0.42  -0.728  0.386 -0.58   1.944  0.425  0.326  0.231 -1.7    0.45  -1.626]
 [ 0.999  0.276 -0.412  0.47   1.203  2.483  1.693 -0.393 -0.514  0.253  0.851  1.979  0.391 -1.124 -0.565]
 [ 0.395  0.669  0.355  1.844  1.508 -1.087  0.444 -1.453 -0.979 -0.715 -0.028 -1.421  0.203  0.429 -0.149]
 [-0.717  0.605  0.424 -0.467  1.664  1.768 -1.275 -0.025 -1.732  0.754  1.037  0.67  -0.171 -0.809 -1.07 ]
 [ 1.628  0.442  0.073 -0.721  0.263  1.139  0.203 -0.762 -0.177  0.789  1.833 -0.364 -1.891 -0.381  0.637]
 [ 0.241 -1.358  0.348  0.59

In [4]:
### testing

# test dimension D_test
D_test = x_test.shape[1]

test_ones_1 = np.ones((1,D_test))
test_ones_2 = np.ones((1,D_test))

# forward pass on test data
test_a_2 = sigmoid(w_2.dot(x_test) + b_2.dot(np.ones((1,x_test.shape[1]))))
test_a_3 = sigmoid(w_3.dot(test_a_2) + b_3.dot(np.ones((1,x_test.shape[1]))))
predictions = np.argmax(test_a_3, axis=0)

num_wrong = np.sum(np.not_equal(y_test, predictions))
test_size = y_test.shape[1]
print(f"{num_wrong} of {test_size} wrong \nAccuracy: {1 - (num_wrong/test_size)}")

919 of 10000 wrong 
Accuracy: 0.9081


In [8]:
test_a_2 = sigmoid(w_2.dot(x_train[:, [0]]) + b_2.dot(np.ones((1,x_train[:, [0]].shape[1]))))
test_a_3 = sigmoid(w_3.dot(test_a_2) + b_3.dot(np.ones((1,x_train[:, [0]].shape[1]))))
predictions = np.argmax(test_a_3, axis=0)
print(predictions)

[5]


In [4]:
# using custom network class
from model import Network

def sigmoid(x): return 1/(1+np.exp(-x))

def sigmoid_prime(x): 
    sig = sigmoid(x)
    return sig * (1-sig)

# define loss & cost function and derivative
def mse_loss(activation, label): return 0.5*(activation-label)**2
def mse_loss_prime(activation, label): return (activation-label)

def cost(loss, activation, label): return np.average(loss(activation, label))
# def cost(loss): return np.average(loss)

network = Network(dims=(784,15,10), activation_funcs = [(sigmoid, sigmoid_prime),(sigmoid, sigmoid_prime)], loss=(mse_loss, mse_loss_prime), cost=cost, seed=1)

network.train(train_data=x_train, labels=y_train, batch_size=1, learning_rate=0.01, epochs=20, verbose=True)


Cost = 0.21236912276027578
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805
Cost = 0.049999999998823805


In [ ]:
# to compare to
import torch
import torch.nn

model = torch.nn.Sequential(
    torch.nn.Linear(784,hidden_dim, dtype=torch.float64),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_dim,10, dtype=torch.float64),
    torch.nn.Sigmoid()
)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

loss_fn = torch.nn.MSELoss()

x_train_tensor = torch.from_numpy(x_train.transpose())
y_train_tensor = torch.from_numpy(y_train.transpose())

for epoch in range(epochs):
    y_pred = model(x_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

x_test_tensor = torch.from_numpy(x_test.transpose())

y_test_t = model(x_test_tensor)
predictions = torch.argmax(y_test_t, dim=1)
